# Visualisation de données : Projet Chocolat

_Réalisé par Barbillon Héléna, Cordeabr Benjamin, Hornberger Théo et Ménestrel Victor_

## Sommaire

* 1- Initialisation
* 2- Quels sont les pourcentages de cacao les plus fréquents ?
* 3- Est-ce que la notation des chocolats est bien répartie ?
* 4-
* 5-
* 6- Comment les gens apprécient-ils le chocolat ?


## Initialisation


In [65]:
import numpy as np
import pandas as pd
import plotly.express as px

# Read the data
df = pd.read_csv('data/chocolate.csv')
print(df.head(3))

# vérifier qu'il ne manque pas de données
print(df.isnull().sum())
#il manque quelques données sur la colonne 'ingredients'

# vérifier qu'il n'y a pas de doublons
print(df.duplicated().sum())


palette = ['#874646', '#f6eddb', '#2b0808', '#6a4227', '#ffc0c0']
palette_degrade =['#71260A', '#96421D' ,'#C06530', '#DA8A56', '#E9D5CC']
background_color = "#fff7ef"
text_color = '#6f1c0e'

    ref company_manufacturer company_location  review_date  \
0  2454                 5150           U.S.A.         2019   
1  2458                 5150           U.S.A.         2019   
2  2454                 5150           U.S.A.         2019   

  country_of_bean_origin specific_bean_origin_or_bar_name cocoa_percent  \
0               Tanzania            Kokoa Kamili, batch 1           76%   
1     Dominican Republic                  Zorzal, batch 1           76%   
2             Madagascar           Bejofo Estate, batch 1           76%   

  ingredients most_memorable_characteristics  rating  
0    3- B,S,C      rich cocoa, fatty, bready    3.25  
1    3- B,S,C         cocoa, vegetal, savory    3.50  
2    3- B,S,C   cocoa, blackberry, full body    3.75  
ref                                  0
company_manufacturer                 0
company_location                     0
review_date                          0
country_of_bean_origin               0
specific_bean_origin_or_bar_name   

## Quels sont les pourcentages de cacao les plus fréquents ?

In [66]:
# percentage of chocolate
data = df.copy()
data['cocoa_percent'] = data['cocoa_percent'].str.replace('%', '').astype(float)
fig = px.box(data, x='cocoa_percent',color_discrete_sequence=palette,points="all",title='Pourcentage de cacao dans le chocolat', labels={'cocoa_percent':'Pourcentage de cacao'})
fig.update_layout( plot_bgcolor=background_color, font_color=text_color)
fig.show()

## Est-ce que la notation du chocolat est une notation bien répartie ?

In [67]:
data = df.copy()
fig = px.histogram(data, x='rating', title='Note des chocolats', labels={'rating':'Note'}, color_discrete_sequence=palette,nbins=15,range_x=[1,4])
fig.update_layout(plot_bgcolor=background_color, font_color=text_color)
fig.show()

On peut voir que les notes ont tendace a être en grande majortié au moins au dessus de 2.5. On peut supposer que les gens aiment le chocolat même si c'est un chocolat de mauvaise qualité.

## Quels sont les pays producteurs de cacao ? Et quels sont les pays qui produisent le plus de chocolat ?

In [68]:
databis = df.copy()
databis['country_of_bean_origin'] = databis['country_of_bean_origin'].apply(lambda x: x if databis['country_of_bean_origin'].value_counts()[x] > 30 else 'Autre pays')

#On retire Blend (correspondant à un mélange) pour ne pas fausser les données
fig=px.pie(databis[databis['country_of_bean_origin'] != 'Blend'], names='country_of_bean_origin', 
           title='Pays d\'origine des fèves de cacao', hole=0.4, color_discrete_sequence=px.colors.sequential.solar)
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.update_layout(margin=dict(l=40, r=20, t=50, b=10), showlegend=False)
fig.update_layout(plot_bgcolor=background_color, font_color=text_color)

fig.update_layout(width=500, height=500)

fig.show()

In [69]:
data = df.copy()
tab=data['country_of_bean_origin'].value_counts()

# Origine des fèves de cacao par pays
fig = px.scatter_geo(tab, locations=tab.index, hover_name=tab.index, locationmode='country names', 
                     size=tab.values, title='Pays producteurs de fèves de cacao' , color=tab.values, 
                     projection="natural earth", color_continuous_scale=palette_degrade)
#projection="orthographic"
fig.update_layout(margin=dict(l=40, r=20, t=50, b=10),coloraxis_colorbar=dict(title='Nombre de chocolats'))
fig.update_traces(marker=dict(line=dict(width=1, color='Black')))
fig.update_layout(font_color=text_color) #couleurs

fig.show()

In [70]:
databis = df.copy()
databis['company_location'] = databis['company_location'].apply(lambda x: x if databis['company_location'].value_counts()[x] > 30 else 'Autre pays')


fig=px.pie(databis, names='company_location', title='Pays importateurs de fèves de cacao', hole=0.4,color_discrete_sequence=px.colors.sequential.solar)

fig.update_traces(textposition='inside', textinfo='percent+label')
fig.update_layout(margin=dict(l=40, r=20, t=50, b=10), showlegend=False)
fig.update_layout(plot_bgcolor=background_color, font_color=text_color)

fig.update_layout(width=500, height=500)

fig.show()

In [71]:
data = df.copy()
tab=data['company_location'].value_counts()

fig = px.scatter_geo(tab, locations=tab.index, hover_name=tab.index, locationmode='country names', size=tab.values,
                      title='Pays importateurs de cacao' , color=tab.values, projection="natural earth", size_max=50,
                      color_continuous_scale=palette_degrade)
#projection="orthographic"
fig.update_layout(margin=dict(l=40, r=20, t=50, b=10),coloraxis_colorbar=dict(title='Nombre de chocolats'))
fig.update_traces(marker=dict(line=dict(width=1, color='Black')))
fig.update_layout(font_color=text_color) #couleurs

fig.show()

On peut conclure que les pays producteurs sont des pays plutôt pauvres en comparaisons aux pays qui importent ces fêves et le transforment en chocolat

## 6 - Comment les gens apprécient-ils le chocolat ?

Tout d'abord nous vérifions l'état de la base de données, notamment, nous vérifions qu'il ne manque pas de données importantes. Nous avons trouvé précédement qu'il manquait des ingrédients. 87 lignes nous pas la donnée "ingrédient" sur 25530. Nous choisissons de les supprimer de la table.  

In [72]:
print(f"Nombre de lignes avant suppression: {df.shape[0]}")
data_cleaned = df.dropna(subset=['ingredients'])
print(f"Nombre de lignes après suppression: {data_cleaned.shape[0]}")

Nombre de lignes avant suppression: 2530
Nombre de lignes après suppression: 2443


Nous voulons étudier les ingrédients et leur nombre. C'est pourquoi nous allons modifier la dataframe. Nous ajoutons des colonnes pour tranformer les données "ingrédients" en format binaire. 

In [73]:
# Décoder les ingrédients
ingredients = {
    'B': 'Beans',
    'S': 'Sugar',
    'S*': 'Sweetener other than sugar',
    'C': 'Cocoa Butter',
    'V': 'Vanilla',
    'L': 'Lecithin',
    'Sa': 'Salt'
}

# Créer une copie indépendante pour éviter les avertissements lors de la modification
data_cleaned2 = data.dropna(subset=['ingredients']).copy()

for ingredient, full_name in ingredients.items():
    data_cleaned2[full_name] = data_cleaned2['ingredients'].apply(lambda x: 1 if ingredient in x else 0)

print(data_cleaned2.head(3))

    ref company_manufacturer company_location  review_date  \
0  2454                 5150           U.S.A.         2019   
1  2458                 5150           U.S.A.         2019   
2  2454                 5150           U.S.A.         2019   

  country_of_bean_origin specific_bean_origin_or_bar_name cocoa_percent  \
0               Tanzania            Kokoa Kamili, batch 1           76%   
1     Dominican Republic                  Zorzal, batch 1           76%   
2             Madagascar           Bejofo Estate, batch 1           76%   

  ingredients most_memorable_characteristics  rating  Beans  Sugar  \
0    3- B,S,C      rich cocoa, fatty, bready    3.25      1      1   
1    3- B,S,C         cocoa, vegetal, savory    3.50      1      1   
2    3- B,S,C   cocoa, blackberry, full body    3.75      1      1   

   Sweetener other than sugar  Cocoa Butter  Vanilla  Lecithin  Salt  
0                           0             1        0         0     0  
1                          

Pour calculer le nombre moyen d'ingrédients nous allons également modifier la dataframe en ajoutant une colonnes correspondant à ce dernier.  

In [74]:
#Nous allons ajouter une nouvelle colonne qui compte le nombre d'ingrédients pour chaque chocolat
data_cleaned2['Num_Ingredients'] = data_cleaned2[['Beans', 'Sugar', 'Sweetener other than sugar', 'Cocoa Butter', 'Vanilla', 'Lecithin', 'Salt']].sum(axis=1)

print(data_cleaned2.head(3))


    ref company_manufacturer company_location  review_date  \
0  2454                 5150           U.S.A.         2019   
1  2458                 5150           U.S.A.         2019   
2  2454                 5150           U.S.A.         2019   

  country_of_bean_origin specific_bean_origin_or_bar_name cocoa_percent  \
0               Tanzania            Kokoa Kamili, batch 1           76%   
1     Dominican Republic                  Zorzal, batch 1           76%   
2             Madagascar           Bejofo Estate, batch 1           76%   

  ingredients most_memorable_characteristics  rating  Beans  Sugar  \
0    3- B,S,C      rich cocoa, fatty, bready    3.25      1      1   
1    3- B,S,C         cocoa, vegetal, savory    3.50      1      1   
2    3- B,S,C   cocoa, blackberry, full body    3.75      1      1   

   Sweetener other than sugar  Cocoa Butter  Vanilla  Lecithin  Salt  \
0                           0             1        0         0     0   
1                        

Nous allons étudier l'évolution du nombre d'ingrédients par chocolat en fonction du temps (années).

In [75]:
# Grouper par année ('review_date') et calculer la moyenne du nombre d'ingrédients et la moyenne des notes
average_values_per_year = data_cleaned2.groupby('review_date').agg({'Num_Ingredients': 'mean', 'rating': 'mean'}).reset_index()

# Renommer les colonnes pour plus de clarté
average_values_per_year.rename(columns={'Num_Ingredients': 'Average_Ingredients', 'rating': 'Average_Rating'}, inplace=True)

# Créer un graphique linéaire pour visualiser l'évolution du nombre d'ingrédients par année
fig = px.line(average_values_per_year, x='review_date', y='Average_Ingredients', title='Évolution du nombre d\'ingrédients par année',
              labels={'review_date': 'Année', 'Average_Ingredients': 'Nombre d\'ingrédients'},color_discrete_sequence=['#2b0808'])

fig.update_layout(plot_bgcolor='#f6eddb', paper_bgcolor='lightgray')
fig.show()

Nous allons étudier l'évolution de la note moyenne des chocolats en fonction du temps (années).

In [76]:
# Créer un graphique linéaire pour visualiser l'évolution de la note moyenne par année
fig = px.line(average_values_per_year, x='review_date', y='Average_Rating', title='Évolution de la note moyenne par année',
              labels={'review_date': 'Année', 'Average_Rating': 'Note moyenne'},color_discrete_sequence=['#2b0808'])

fig.update_layout(plot_bgcolor='#f6eddb', paper_bgcolor='lightgray')
fig.show()

Conclusion : cela nous fait penser à une possible corrélation entre le la note moyenne des chocolats et leurs nombre d'ingrédients.
Nous allons, alors, effectuer une matrice de corrélation entre la note moyenne et le nombre d'ingrédients des chocolats.

In [77]:
# Calculer la matrice de corrélation
correlation_matrix = average_values_per_year[['Average_Ingredients', 'Average_Rating']].corr()

# Renommage des colonnes dans la matrice de corrélation pour des étiquettes plus descriptives
correlation_matrix = correlation_matrix.rename(index={'Average_Ingredients': 'Nombre moyen d\'ingrédients', 'Average_Rating': 'Note Moyenne'},
                                               columns={'Average_Ingredients': 'Nombre moyen d\'ingrédients', 'Average_Rating': 'Note Moyenne'})
# Définir une échelle de couleurs personnalisée en marron
brown_scale = [
    [0, 'rgb(255,248,220)'],  # Cornsilk color
    [0.5, 'rgb(210,180,140)'],  # Tan color
    [1, '#96421D']  # Brown color
]
# imshow avec plotly
fig = px.imshow(correlation_matrix, color_continuous_scale=brown_scale, text_auto=True, 
                title='Matrice de corrélation entre le nombre d\'ingrédients et la note moyenne')
# Mise à jour du titre et de l'étiquette de la barre de couleurs
fig.update_layout(
    coloraxis_colorbar=dict(
        title='Coefficient de corrélation de Pearson',
    )
)
fig.show()

 Conclusion : Les personnes attribuent des notes de plus en plus élevées, ce qui peut suggérer que les chocolats deviennent de plus en plus savoureux ou que les gens deviennent plus généreux dans leurs évaluations. Par ailleurs, nous avons remarqué une bonne corrélation entre le nombre d'ingrédients et la note moyenne des chocolats. Attention, néanmoins, la corrélation n'implique par la causalité mais cette dernière pourrait expliquer certaines choses.